In [1]:
# Import

import os
from pathlib import Path
from shapely.geometry import Point
import numpy as np
import pandas as pd
import geopandas as gpd
import leafmap.foliumap as leafmap

In [ ]:
# Pathing

current = Path.cwd()
parent = current.resolve().parent / 'data'
os.chdir(parent)
os.getcwd()

In [3]:
# Load

NTA10 = gpd.read_file("NTA10.geojson")
NTA20 = gpd.read_file("NTA20.geojson")
NYC_CenB2010 = gpd.read_file("NYC_CenB_2010_Geo.geojson")
NYC_CT2010 = gpd.read_file("NYC_CT2010.geojson")
NYC_CT2010_Crosswalk = gpd.read_file("NYC_CT2010_Crosswalk.geojson")
CommunityDistricts = gpd.read_file("Community_Districts.geojson")

#FC_BBL = gpd.read_file("FC_SHD_08-29\FC_Subsidized_Housing_Database_2024-08-29\FC_SHD_bbl_analysis_2024-08-29.csv")
#FC_Subsidy = gpd.read_file("FC_SHD_08-29\FC_Subsidized_Housing_Database_2024-08-29\FC_SHD_subsidy_analysis_2024-08-29.csv")
FC_421a_geo = gpd.read_file("FC_421a.geojson")
FC_421a_att = gpd.read_file("FC_421_properties.geojson")
FC_421a_ResUnits_CT_Geo_Diff =  gpd.read_file("FC_421a_ResUnits_CT_Geo_Diff.geojson")

#FC_Sub_Subsidy = np.unique(FC_Subsidy['sub_subsidy_name'])

In [4]:
# 421a Wrangling

FC_421a_geo = FC_421a_geo[['bbl',
                        'standard_address',
                        'boro_id',
                        'cd_id',
                        'tract_10',
                        'start_421a',
                        'end_421a',
                        'prog_421a_aff',
                        'start_421a_aff', 
                        'end_421a_aff',
                        'geometry'
                        ]]

FC_421a_att.drop(columns={'geometry', 'tract_10'}, inplace=True)

NYC_CT2010_Crosswalk = NYC_CT2010_Crosswalk[['ntacode',
                                             'ntaname',
                                             'boro_name',
                                             'puma',
                                             'ct2010',
                                             'boro_code']]

NYC_CT2010_Crosswalk['FIPS'] = ''
NYC_CT2010_Crosswalk.loc[NYC_CT2010_Crosswalk['boro_code'] == '1', 'FIPS'] = '061'
NYC_CT2010_Crosswalk.loc[NYC_CT2010_Crosswalk['boro_code'] == '2', 'FIPS'] = '005'
NYC_CT2010_Crosswalk.loc[NYC_CT2010_Crosswalk['boro_code'] == '3', 'FIPS'] = '047'
NYC_CT2010_Crosswalk.loc[NYC_CT2010_Crosswalk['boro_code'] == '4', 'FIPS'] = '081'
NYC_CT2010_Crosswalk.loc[NYC_CT2010_Crosswalk['boro_code'] == '5', 'FIPS'] = '085'
NYC_CT2010_Crosswalk['GEOID10'] = '36' + NYC_CT2010_Crosswalk['FIPS'] + NYC_CT2010_Crosswalk['ct2010']

FC_421a_merged = pd.merge(left=FC_421a_att,
                          right=FC_421a_geo,
                          how='left',
                          on='bbl')

FC_421a_merged = gpd.GeoDataFrame(FC_421a_merged, crs='EPSG:4326')
FC_421a_merged['res_units'] = FC_421a_merged['res_units'].astype(int)

FC_421a_merged.rename(columns={'cd_id': 'boro_cd'}, inplace=True)
FC_421a_merged.rename(columns={'tract_10': 'GEOID10'}, inplace=True)


#FC_421a_merged['tract_10'] = FC_421a_merged['tract_10'].str.slice(start=4)
#FC_421a_merged_short = FC_421a_merged[['standard_address', 'bbl', 'geometry']] 

In [5]:
# CD Agg

FC_421a_ResBuild_CD = FC_421a_merged.groupby('boro_cd').size().reset_index(name='BuildCount')
FC_421a_ResUnits_CD = FC_421a_merged.groupby('boro_cd')['res_units'].sum().reset_index(name='UnitCount')

# CT Agg

FC_421a_ResBuild_CT = FC_421a_merged.groupby('GEOID10').size().reset_index(name='BuildCount')
FC_421a_ResUnits_CT = FC_421a_merged.groupby('GEOID10')['res_units'].sum().reset_index(name='UnitCount')

In [ ]:
# CD Geo

FC_421a_ResBuild_CD_Geo = pd.merge(left=CommunityDistricts,
                                right=FC_421a_ResBuild_CD,
                                how='left',
                                on='boro_cd')
FC_421a_ResBuild_CD_Geo = FC_421a_ResBuild_CD_Geo.dropna()

FC_421a_ResUnits_CD_Geo = pd.merge(left=CommunityDistricts,
                                right=FC_421a_ResUnits_CD,
                                how='left',
                                on='boro_cd')
FC_421a_ResUnits_CD_Geo = FC_421a_ResUnits_CD_Geo.dropna()

# CT Geo

FC_421a_ResBuild_CT_Geo = pd.merge(left=NYC_CT2010,
                                right=FC_421a_ResBuild_CT,
                                how='left',
                                on='GEOID10')
FC_421a_ResBuild_CT_Geo.dropna(inplace=True)

FC_421a_ResUnits_CT_Geo = pd.merge(left=NYC_CT2010,
                                right=FC_421a_ResUnits_CT,
                                how='left',
                                on='GEOID10')
FC_421a_ResUnits_CT_Geo['UnitCount'] = FC_421a_ResUnits_CT_Geo['UnitCount'].fillna(0)

#FC_421a_ResUnits_CT_Geo['boro_ct2010'] = FC_421a_ResUnits_CT_Geo['GEOID10'].str.slice(start=2)

FC_421a_ResUnits_NTA_fromCT = pd.merge(left=FC_421a_ResUnits_CT_Geo,
                                right=NYC_CT2010_Crosswalk,
                                how='left',
                                on='GEOID10'
                                )
FC_421a_ResUnits_NTA_fromCT = FC_421a_ResUnits_NTA_fromCT.dropna()

# NTA Agg

FC_421a_ResBuild_NTA10 = FC_421a_ResUnits_NTA_fromCT.groupby('ntacode').size().reset_index(name='BuildCount')
FC_421a_ResUnits_NTA10 = FC_421a_ResUnits_NTA_fromCT.groupby('ntacode')['UnitCount'].sum().reset_index(name='NTA_UnitCount')

FC_421a_ResUnits_NTA10_Geo = pd.merge(left=FC_421a_ResUnits_NTA10,
                                      right=NTA10,
                                      how='left',
                                      on='ntacode'
                                      )

#FC_421a_ResUnits_NTA10_Park_Geo = FC_421a_ResUnits_NTA10_Geo.loc[:, ~FC_421a_ResUnits_NTA10_Geo['ntaname'].str.startswith('park')]

FC_421a_ResUnits_NTA10_Geo = gpd.GeoDataFrame(data=FC_421a_ResUnits_NTA10_Geo,
                                              geometry='geometry',
                                              crs='EPSG:4326')

print(FC_421a_ResUnits_NTA10_Geo['NTA_UnitCount'].sort_values(ascending=False))

In [7]:
# Export

#FC_421a_geo.to_file("FC_421a_geo.geojson", driver='GeoJSON')
#FC_421a_merged.to_file("FC_421a_merged.geojson", driver='GeoJSON')
#FC_421a_ResUnits_NTA10_Geo.to_file(filename="FC_421a_ResUnits_NTA10_Geo.geojson", driver="GeoJSON")
#FC_421a_ResUnits_CT_Geo.to_file(filename="FC_421a_ResUnits_CT_Geo.geojson", driver="GeoJSON")


In [8]:
# Geometry 

# FC_BBL['geometry'] = [Point(x, y) for x, y in zip(FC_BBL['longitude'], FC_BBL['latitude'])]
# FC_BBL.set_crs(epsg=4326, inplace=True)

# FC_421a = FC_BBL[['bbl',
#                'standard_address',
#                'boro_id',
#                'cd_id',
#                'tract_10',
#                'start_421a',
#                'end_421a',
#                'prog_421a_aff',
#                'start_421a_aff', 
#                'end_421a_aff',
#                'geometry']]
# Datetime_columns = ['start_421a',
#                     'end_421a',
#                     'prog_421a_aff',
#                     'start_421a_aff', 
#                     'end_421a_aff']

# #pd.to_datetime(FC_421a[[Datetime_columns]], errors='coerce')
# #FC_421a.loc
# #FC_421a.to_file(filename='FC_421a.geojson', driver='GeoJSON')

In [ ]:
# Map Template

m = leafmap.Map(#center=[40.7248387,-73.9775537]
                zoom=12, tiles="CartoDB.Positron")
m.add_data(FC_421a_ResUnits_NTA10_Geo,
           column='NTA_UnitCount',
           scheme='NaturalBreaks',
           k=5,
           add_legend=True,
           info_mode='on_hover')
m